#  Data collection & Munging

In dit notebook zal the ruwe datasets omgezet worden naar makkelijk bewerkbare data in een Pandas dataframe.

#### De XML dataset
In de XML dataset staat wel ontwikkeling zich afspelen op het veld. Je kunt denken aan bal winst of een goal. Ook wordt er bijghouden waar de ontwikkeling plaats vind. Bijvoorbeeld L2 of R1 (Zie afbeelding).
<img src="./voetbalveld-indeling.png" width=300>

##### XML structuur
De XML is als volgt op gebouwd:
```XML
<ALL_INSTANCES>
    <instance>
        <ID>1</ID>
        <start>4.3110428970</start>
        <end>40.9711876731</end>
        <code>TEG</code>
        <label>
            <group>Naar zone</group>
            <text>M2 TEG</text>
        </label>
    </instance>
</ALL_INSTANCES>
```
Er komen in ```<ALL_INSTANCES>``` meerdere instances voor.

Om een xml intelezen hebben van de XML library ElementTree nodig. Hiermee kunnen we makkelijk alle data uit de XML halen. We hebben ook pandas nodig omdat we pandas willen gebruiken voor de analyse.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

De eerste stap zal zijn om de XML dataset om te zetten naar een pandas dataframe. Hiervoor maken we een functie waarin we de root van de dataset kunnen meegeven. We maken er een functie van omdat we dit voor meerdere datasets willen doen. 
<br />
<br />
In de functie zullen door alle childeren van de root gaan (we zitten dan in ```<instance>```) en daarvan de subchilderen. voor elke subchild checken we of het een label tag is. Dit doen we omdat de label tag meer subchilderen heeft. Als dat niet zo is dan voegen we de tekst van de tag toe aan de record en als de tag naam niet in de header list zit voegen we die ook toe. Als de tag wel een label loopen we door de childeren van die label tag. Een instance tag kan een verschillende hoeveelheid label tags bevatten. Daarom moeten we meerdere headers toevoegen voor de label tags. 
<br />
<br />
Als dat allemaal gedaan is krijgen we een mooie pandas dataset waar we mee kunnen werken.

In [26]:
tree = ET.parse('datasets/new-xml/20160410 FC Utrecht - NEC-new.xml')

root = tree.getroot()

def xmlRootToDataframe(root):
    all_instances = root[1]
    all_records = []
    headers = []
    for i, child in enumerate(all_instances):
        current_label = 0
        record = []
        for subchild in child:
            label_name = ''
            if subchild.tag == 'label':
                subrecord = []
                label_name = subchild.tag+'.'+str(current_label)
                if label_name not in headers:
                        headers.append(label_name)
                for subsubchild in subchild:
                    subrecord.append(subsubchild.text)
                current_label += 1
                record.append(subrecord)
            else:
                record.append(subchild.text)
            if subchild.tag not in headers and subchild.tag not in 'label':
                headers.append(subchild.tag)

        all_records.append(record)

    return pd.DataFrame(all_records, columns=headers)

df = xmlRooToDataframe(root)
df.head()

,ID,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
0,1,4.3110428970,40.9711876731,TEG,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
1,2,6.2150178417,15.1347948447,M2 TEG,"[Naar zone, L2 TEG]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3,6.3110428970,40.9711876731,Start Jaimy,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
3,4,6.3110428970,40.9711876731,Verdedigen,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,5,8.2200000000,14.0400000000,AFTRAP WB,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [27]:
df.dtypes

ID          object
start       object
end         object
code        object
label.0     object
label.1     object
label.2     object
label.3     object
label.4     object
label.5     object
label.6     object
label.7     object
label.8     object
label.9     object
label.10    object
label.11    object
label.12    object
label.13    object
label.14    object
label.15    object
label.16    object
label.17    object
label.18    object
label.19    object
label.20    object
label.21    object
label.22    object
label.23    object
label.24    object
label.25    object
             ...  
label.42    object
label.43    object
label.44    object
label.45    object
label.46    object
label.47    object
label.48    object
label.49    object
label.50    object
label.51    object
label.52    object
label.53    object
label.54    object
label.55    object
label.56    object
label.57    object
label.58    object
label.59    object
label.60    object
label.61    object
label.62    object
label.63    

De datatypes van alle kolommen zijn nu objecten. Dit is niet idiaal en zal later beperkingen opleveren. laten we start en end numeric maken. We maken de id kolom ook gelijk de index.
<br />
<br />
We beginnen de index veranderen daarna zetten we de datatypes om.

In [28]:
df = df.set_index('ID')
df.index.name = None

df[['start','end']] = df[['start','end']].apply(pd.to_numeric)
df.code = df.code.astype('category')

print(df[['start','end', 'code']].dtypes)

start     float64
end       float64
code     category
dtype: object
